# 内置中间件

> 用于常见Agent用例的预构建中间件

LangChain提供了用于常见用例的预构建中间件。每个中间件都是生产就绪的，并且可以根据您的特定需求进行配置。

## 提供商无关的中间件

以下中间件适用于任何LLM提供商：

| 中间件                              | 描述                                     |
| ----------------------------------- | ---------------------------------------- |
| [摘要生成](#summarization)          | 在接近token限制时自动摘要对话历史。       |
| [人工介入](#human-in-the-loop)      | 暂停执行以获取工具调用的人工批准。       |
| [模型调用限制](#model-call-limit)   | 限制模型调用次数以防止过度成本。         |
| [工具调用限制](#tool-call-limit)    | 通过限制调用次数来控制工具执行。         |
| [模型回退](#model-fallback)         | 当主模型失败时自动回退到替代模型。       |
| [PII检测](#pii-detection)           | 检测和处理个人身份信息(PII)。            |
| [待办列表](#to-do-list)             | 为Agent配备任务规划和跟踪能力。          |
| [LLM工具选择器](#llm-tool-selector) | 在调用主模型之前使用LLM选择相关工具。    |
| [工具重试](#tool-retry)             | 使用指数退避自动重试失败的工具调用。     |
| [模型重试](#model-retry)            | 使用指数退避自动重试失败的模型调用。     |
| [LLM工具模拟器](#llm-tool-emulator) | 使用LLM模拟工具执行以进行测试。          |
| [上下文编辑](#context-editing)      | 通过修剪或清除工具使用来管理对话上下文。 |
| [Shell工具](#shell-tool)            | 为Agent暴露持久shell会话以执行命令。     |
| [文件搜索](#file-search)            | 提供文件系统文件的Glob和Grep搜索工具。   |

### 摘要生成（Summarization）

在接近token限制时自动摘要对话历史，保留最近的消息同时压缩较旧的上下文。摘要生成在以下场景中非常有用：

* 超过上下文窗口的长时间运行对话
* 具有大量历史记录的多轮对话。
* 需要保留完整对话上下文的应用程序。

> **API参考：** [`SummarizationMiddleware`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.SummarizationMiddleware)

```python
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware

agent = create_agent(
    model="gpt-4o",
    tools=[your_weather_tool, your_calculator_tool],
    middleware=[
        SummarizationMiddleware(
            model="gpt-4o-mini",
            trigger=("tokens", 4000),
            keep=("messages", 20),
        ),
    ],
)
```

#### 配置选项

`trigger`和`keep`的`fraction`条件（如下所示）依赖于聊天模型的配置文件数据，当`langchain>=1.1`。如果数据不可用，请使用其他条件或手动指定：
```python
from langchain.chat_models import init_chat_model

custom_profile = {
"max_input_tokens": 100_000,
# ...
}
model = init_chat_model("gpt-4o", profile=custom_profile)
```

- "model" type="string | BaseChatModel" required  
用于生成摘要的模型。可以是模型标识符字符串（例如`'openai:gpt-4o-mini'`）或`BaseChatModel`实例。有关更多信息，请参阅[`init_chat_model`](https://reference.langchain.com/python/langchain/models/#langchain.chat_models.init_chat_model\(model\))。


- "trigger" type="ContextSize | list[ContextSize] | None
触发摘要生成的条件。可以是：  
  * 单个[`ContextSize`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.summarization.ContextSize)字典（所有属性必须满足 - AND逻辑）
  * [`ContextSize`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.summarization.ContextSize)字典列表（任何一个条件必须满足 - OR逻辑）

  每个条件可以包括：

  * `fraction` (float)：模型上下文大小的比例(0-1)
  * `tokens` (int)：绝对token数量
  * `messages` (int)：消息数量

  每个条件必须至少指定一个属性。如果未提供，摘要生成将不会自动触发。

  有关更多信息，请参阅[`ContextSize`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.summarization.ContextSize)的API参考。


- "keep" type="ContextSize" default="{messages: 20}
摘要后保留多少上下文。指定以下之一：  
  * `fraction` (float)：保留的模型上下文大小比例(0-1)
  * `tokens` (int)：保留的绝对token数量
  * `messages` (int)：保留的最近消息数量

  有关更多信息，请参阅[`ContextSize`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.summarization.ContextSize)的API参考。

- "token_counter" type="function  
自定义token计数函数。默认为基于字符的计数。

- "summary_prompt" type="string  
摘要生成的自定义提示词模板。如果未指定，则使用内置模板。模板应包含`{messages}`占位符，对话历史将插入其中。

- "trim_tokens_to_summarize" type="number" default="4000  
生成摘要时包含的最大token数量。在摘要生成之前，消息将被裁剪以适应此限制。

- "summary_prefix" type="string  
添加到摘要消息的前缀。如果未提供，则使用默认前缀。

- "max_tokens_before_summary" type="number" deprecated  
**已弃用：** 使用`trigger: {"tokens": value}`代替。触发摘要生成的token阈值。

- "messages_to_keep" type="number" deprecated  
**已弃用：** 使用`keep: {"messages": value}`代替。保留的最近消息数量。

#### 完整示例
摘要中间件监控消息的token数量，并在达到阈值时自动摘要旧的消息。

**触发条件**控制何时运行摘要生成：

* 单个条件对象（所有属性必须满足 - AND逻辑）
* 条件数组（任何一个条件满足即可 - OR逻辑）
* 每个条件可以使用`fraction`（模型上下文大小的比例）、`tokens`（绝对数量）或`messages`（消息数量）

**保留条件**控制保留多少上下文（指定以下之一）：

* `fraction` - 保留的模型上下文大小比例
* `tokens` - 保留的绝对token数量
* `messages` - 保留的最近消息数量

```python
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware

# 单个条件：如果token >= 4000 且消息 >= 10 则触发
agent = create_agent(
    model="gpt-4o",
    tools=[your_weather_tool, your_calculator_tool],
    middleware=[
        SummarizationMiddleware(
            model="gpt-4o-mini",
            trigger=[("tokens", 4000), ("messages", 10)],
            keep=("messages", 20),
        ),
    ],
)

# 多个条件
agent2 = create_agent(
    model="gpt-4o",
    tools=[your_weather_tool, your_calculator_tool],
    middleware=[
        SummarizationMiddleware(
            model="gpt-4o-mini",
            trigger=[
                ("tokens", 3000),
                ("messages", 6),
            ],
            keep=("messages", 20),
        ),
    ],
)

# 使用比例限制
agent3 = create_agent(
    model="gpt-4o",
    tools=[your_weather_tool, your_calculator_tool],
    middleware=[
        SummarizationMiddleware(
            model="gpt-4o-mini",
            trigger=("fraction", 0.8),
            keep=("fraction", 0.3),
        ),
    ],
)
```

### 人工介入

暂停Agent执行以获取工具调用的人工批准、编辑或拒绝，然后再执行。人工介入在以下场景中非常有用：

* 需要人工批准的高风险操作（例如数据库写入、金融交易）
* 需要人工监督的合规工作流
* 人工反馈指导Agent的长时间运行对话

> **API参考：** [`HumanInTheLoopMiddleware`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.HumanInTheLoopMiddleware)


人工介入中间件需要[检查点](https://docs.langchain.com/oss/python/langgraph/persistence#checkpoints)来维护跨中断的状态。

```python
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver


def read_email_tool(email_id: str) -> str:
    """模拟函数：通过ID读取电子邮件。"""
    return f"Email content for ID: {email_id}"

def send_email_tool(recipient: str, subject: str, body: str) -> str:
    """模拟函数：发送电子邮件。"""
    return f"Email sent to {recipient} with subject '{subject}'"

agent = create_agent(
    model="gpt-4o",
    tools=[read_email_tool, send_email_tool],
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email_tool": {
                    "allowed_decisions": ["approve", "edit", "reject"],
                },
                "read_email_tool": False,
            }
        ),
    ],
)
```

> 有关完整示例、配置选项和集成模式，请参阅[人工介入文档](https://docs.langchain.com/oss/python/langchain/human-in-the-loop)。

> 观看此[视频指南](https://www.youtube.com/watch?v=SpfT6-YAVPk)演示人工介入中间件行为。

### 模型调用限制

限制模型调用次数以防止无限循环或成本过度。模型调用限制在以下场景中非常有用：

* 防止失控Agent进行过多API调用
* 在生产部署时强制执行成本控制
* 在确定的调用预算内测试Agent行为

> **API参考：** [`ModelCallLimitMiddleware`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.ModelCallLimitMiddleware)

```python
from langchain.agents import create_agent
from langchain.agents.middleware import ModelCallLimitMiddleware

agent = create_agent(
    model="gpt-4o",
    tools=[],
    middleware=[
        ModelCallLimitMiddleware(
            thread_limit=10,
            run_limit=5,
            exit_behavior="end",
        ),
    ],
)
```

> 观看此[视频指南](https://www.youtube.com/watch?v=nJEER0uaNkE)演示模型调用限制中间件行为。

配置选项
- "thread_limit" type="number  
  线程中所有运行的模型调用最大次数。默认为无限制。
  
- "run_limit" type="number  
  单次调用时模型调用最大次数。默认为无限制。
  
- "exit_behavior" type="string" default="end  
  达到限制时的行为。选项：`'end'`（终止）或`'error'`（抛出异常）


### 工具调用限制

通过限制工具调用次数来控制Agent执行，可以全局限制所有工具或特定工具。工具调用限制在以下场景中非常有用：

* 避免对昂贵外部API的过度调用
* 限制Web搜索或数据库查询
* 对特定工具使用强制速率限制
* 防止失控Agent循环

> **API参考：** [`ToolCallLimitMiddleware`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.ToolCallLimitMiddleware)

#### 示例代码
```python
from langchain.agents import create_agent
from langchain.agents.middleware import ToolCallLimitMiddleware

agent = create_agent(
    model="gpt-4o",
    tools=[search_tool, database_tool],
    middleware=[
        # 全局限制
        ToolCallLimitMiddleware(thread_limit=20, run_limit=10),
        # 工具特定限制
        ToolCallLimitMiddleware(
            tool_name="search",
            thread_limit=5,
            run_limit=3,
        ),
    ],
)
```

> 观看此[视频指南](https://www.youtube.com/watch?v=6gYlaJJ8t0w)演示工具调用限制中间件行为。

#### 配置选项
- "tool_name" type="string  
  要限制的特定工具名称。如果未提供，限制适用于**全局所有工具**。
  
- "thread_limit" type="number  
  线程中所有运行的工具调用最大次数（对话）。在具有相同线程ID的多次调用中持续存在。需要检查点来维护状态。`None`表示无线程限制。
  
- "run_limit" type="number  
  单次调用的工具调用最大次数（一个用户消息→响应周期）。每次新用户消息时重置。`None`表示无运行限制。

  **注意：** 必须指定`thread_limit`或`run_limit`中的至少一个。

- "exit_behavior" type="string" default="continue
  达到限制时的行为：  
  * `'continue'`（默认）- 使用错误消息阻止超出的工具调用，让其他工具和模型继续。模型根据错误消息决定何时结束。
  * `'error'` - 抛出`ToolCallLimitExceededError`异常，立即停止执行
  * `'end'` - 立即停止执行，为超出的工具调用生成`ToolMessage`和AI消息。仅在限制单个工具时有效；如果其他工具有待处理调用，则抛出`NotImplementedError`。
  
#### 完整示例
  使用以下方式指定限制：  
  * **线程限制** - 对话中所有运行的最大调用次数（需要检查点）
  * **运行限制** - 单次调用的最大调用次数（每次轮次重置）

  退出行为：  
  * `'continue'`（默认）- 使用错误消息阻止超出的调用，Agent继续
  * `'error'` - 立即抛出异常
  * `'end'` - 使用ToolMessage + AI消息停止（仅限单工具场景）

```python
from langchain.agents import create_agent
from langchain.agents.middleware import ToolCallLimitMiddleware

global_limiter = ToolCallLimitMiddleware(thread_limit=20, run_limit=10)
search_limiter = ToolCallLimitMiddleware(tool_name="search", thread_limit=5, run_limit=3)
database_limiter = ToolCallLimitMiddleware(tool_name="query_database", thread_limit=10)
strict_limiter = ToolCallLimitMiddleware(tool_name="scrape_webpage", run_limit=2, exit_behavior="error")

agent = create_agent(
    model="gpt-4o",
    tools=[search_tool, database_tool, scraper_tool],
    middleware=[global_limiter, search_limiter, database_limiter, strict_limiter],
)
```

### 模型回退
当主模型出现故障时，自动切换到备用模型。模型切换在以下方面很有用：
- 构建能够应对模型中断的具有韧性的智能体。
- 通过切换到更便宜的模型来优化成本。
- 在OpenAI、Anthropic等提供商之间实现冗余。

> > **API参考：** [`ModelFallbackMiddleware`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.ModelFallbackMiddleware)

#### 示例代码
```python
from langchain.agents import create_agent
from langchain.agents.middleware import ModelFallbackMiddleware

agent = create_agent(
    model="gpt-4o",
    tools=[],
    middleware=[
        ModelFallbackMiddleware(
            "gpt-4o-mini",
            "claude-3-5-sonnet-20241022",
        ),
    ],
)
```

> 观看此[视频指南](https://www.youtube.com/watch?v=8rCRO0DUeIM)演示模型回退中间件行为。

#### 配置选项
- "first_model" type="string | BaseChatModel" required  
  当主模型失败时尝试的第一个回退模型。可以是模型标识符字符串（例如`'openai:gpt-4o-mini'`）或`BaseChatModel`实例。
  
- "*additional_models" type="string | BaseChatModel  
  如果先前模型失败，按顺序尝试的备用回退模型


### PII检测
使用可配置策略检测和处理对话中的个人身份信息(PII)。PII检测在以下场景中非常有用：  
* 具有合规要求的医疗保健和金融应用
* 需要清理日志的客户服务Agent
* 任何处理敏感用户数据的应用

> **API参考：** [`PIIMiddleware`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.PIIMiddleware)

```python
from langchain.agents import create_agent
from langchain.agents.middleware import PIIMiddleware

agent = create_agent(
    model="gpt-4o",
    tools=[],
    middleware=[
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        PIIMiddleware("credit_card", strategy="mask", apply_to_input=True),
    ],
)
```

#### 自定义PII类型
你可以通过提供一个`detector`参数来创建自定义的个人身份信息（PII）类型。这使你能够检测超出内置类型范围的、特定于你的使用场景的模式。  

创建自定义检测器的三种方式（代码）：  
1. **正则表达式模式字符串** - 简单的模式匹配
2. **自定义函数** - 具有验证的复杂检测逻辑

```python
from langchain.agents import create_agent
from langchain.agents.middleware import PIIMiddleware
import re


# 方法1：正则表达式模式字符串
agent1 = create_agent(
    model="gpt-4o",
    tools=[],
    middleware=[
        PIIMiddleware(
            "api_key",
            detector=r"sk-[a-zA-Z0-9]{32}",
            strategy="block",
        ),
    ],
)

# 方法2：编译的正则表达式模式
agent2 = create_agent(
    model="gpt-4o",
    tools=[],
    middleware=[
        PIIMiddleware(
            "phone_number",
            detector=re.compile(r"\+?\d{1,3}[\s.-]?\d{3,4}[\s.-]?\d{4}"),
            strategy="mask",
        ),
    ],
)

# 方法3：自定义检测器函数
def detect_ssn(content: str) -> list[dict[str, str | int]]:
    """检测SSN并进行验证。

    返回包含'text'、'start'和'end'键的字典列表。
    """
    import re
    matches = []
    pattern = r"\d{3}-\d{2}-\d{4}"
    for match in re.finditer(pattern, content):
        ssn = match.group(0)
        # 验证：前3位数字不应为000、666或900-999
        first_three = int(ssn[:3])
        if first_three not in [0, 666] and not (900 <= first_three <= 999):
            matches.append({
                "text": ssn,
                "start": match.start(),
                "end": match.end(),
            })
    return matches

agent3 = create_agent(
    model="gpt-4o",
    tools=[],
    middleware=[
        PIIMiddleware(
            "ssn",
            detector=detect_ssn,
            strategy="hash",
        ),
    ],
)
```

**自定义检测器函数返回格式：**
检测器函数必须接受字符串（内容）并返回匹配项：  
返回包含`text`、`start`和`end`键的字典列表。

```python
def detector(content: str) -> list[dict[str, str | int]]:
    return [
        {"text": "matched_text", "start": 0, "end": 12},
        # ... more matches
    ]
```

#### 配置选项

# 配置项
| 参数字段 | 类型 | 必填项 | 默认值 | 说明 |
|----------|------|--------|--------|------|
| `pii_type` | 字符串 | 是 | - | 待检测的个人身份信息（PII）类型，可为内置类型（`email`、`credit_card`、`ip`、`mac_address`、`url`）或自定义类型名称 |
| `strategy` | 字符串 | 否 | `redact` | 检测到个人身份信息后的处理策略，可选值如下：<br>- `'block'` - 检测到即抛出异常<br>- `'redact'` - 替换为 `[REDACTED_TYPE]`<br>- `'mask'` - 部分脱敏（例如 `****-****-****-1234`）<br>- `'hash'` - 替换为确定性哈希值 |
| `detector` | 函数 \| 正则表达式 | 否 | - | 自定义检测函数或正则表达式。若未配置，则使用该个人身份信息类型对应的内置检测器 |
| `apply_to_input` | 布尔值 | 否 | `True` | 在调用模型前检查用户输入消息 |
| `apply_to_output` | 布尔值 | 否 | `False` | 在调用模型后检查人工智能输出消息 |
| `apply_to_tool_results` | 布尔值 | 否 | `False` | 在工具执行后检查工具返回结果消息 |  

### 待办列表

为Agent配备任务规划和跟踪能力，允许Agent将复杂查询分解为可管理的步骤。待办列表在以下场景中非常有用：

* 需要多步骤规划的项目管理
* 需要结构化方法的复杂问题解决
* 需要进度跟踪的长时间运行任务

> **API参考：** [`ToDoListMiddleware`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.ToDoListMiddleware)

```python
from langchain.agents import create_agent
from langchain.agents.middleware import ToDoListMiddleware

agent = create_agent(
    model="gpt-4o",
    tools=[search_tool, calculator_tool],
    middleware=[
        ToDoListMiddleware(
            model="gpt-4o-mini",
            max_tasks=10,
            auto_prioritize=True,
        ),
    ],
)
```

配置选项
  "model" type="string | BaseChatModel" required
    用于任务规划和分解的模型。可以是模型标识符字符串（例如`'openai:gpt-4o-mini'`）或`BaseChatModel`实例。
  

  "max_tasks" type="number" default="10
    待办列表中允许的最大任务数。
  

  "auto_prioritize" type="boolean" default="true
    是否自动按优先级对任务排序。
  

  "task_prompt" type="string
    任务规划的自定义提示模板。
  

  "update_prompt" type="string
    任务更新的自定义提示模板。
  


### LLM工具选择器

在调用主模型之前使用LLM选择相关工具，减少不必要工具调用的开销。LLM工具选择器在以下场景中非常有用：

* 具有大量工具的大型工具集
* 需要优化工具选择的性能关键应用
* 减少不必要API调用的成本敏感部署

> **API参考：** [`LLMToolSelectorMiddleware`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.LLMToolSelectorMiddleware)

```python
from langchain.agents import create_agent
from langchain.agents.middleware import LLMToolSelectorMiddleware

agent = create_agent(
    model="gpt-4o",
    tools=[search_tool, calculator_tool, database_tool],
    middleware=[
        LLMToolSelectorMiddleware(
            selector_model="gpt-4o-mini",
            max_tools=3,
            confidence_threshold=0.7,
        ),
    ],
)
```

配置选项
  "selector_model" type="string | BaseChatModel" required
    用于工具选择的模型。可以是模型标识符字符串（例如`'openai:gpt-4o-mini'`）或`BaseChatModel`实例。
  

  "max_tools" type="number" default="5
    选择器可以考虑的最大工具数。
  

  "confidence_threshold" type="float" default="0.5
    工具选择的最小置信度阈值。
  

  "selector_prompt" type="string
    工具选择的自定义提示模板。
  


### 工具重试

使用指数退避自动重试失败的工具调用，提高Agent的弹性。工具重试在以下场景中非常有用：

* 处理临时网络故障或API限制
* 构建对间歇性故障具有弹性的Agent
* 需要高可用性的生产部署

> **API参考：** [`ToolRetryMiddleware`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.ToolRetryMiddleware)

```python
from langchain.agents import create_agent
from langchain.agents.middleware import ToolRetryMiddleware

agent = create_agent(
    model="gpt-4o",
    tools=[api_tool, database_tool],
    middleware=[
        ToolRetryMiddleware(
            max_retries=3,
            backoff_factor=2,
            retryable_errors=["Timeout", "RateLimit"],
        ),
    ],
)
```

配置选项
  "max_retries" type="number" default="3
    失败工具调用的最大重试次数。
  

  "backoff_factor" type="float" default="2
    重试之间的指数退避因子。
  

  "retryable_errors" type="list[string]
    应重试的错误类型列表。如果未指定，重试所有错误。
  

  "tool_name" type="string
    要应用重试的特定工具名称。如果未提供，重试适用于所有工具。
  


### 模型重试

使用指数退避自动重试失败的模型调用，提高Agent的弹性。模型重试在以下场景中非常有用：

* 处理临时模型API故障或限制
* 构建对间歇性模型故障具有弹性的Agent
* 需要高可用性的生产部署

> **API参考：** [`ModelRetryMiddleware`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.ModelRetryMiddleware)

```python
from langchain.agents import create_agent
from langchain.agents.middleware import ModelRetryMiddleware

agent = create_agent(
    model="gpt-4o",
    tools=[],
    middleware=[
        ModelRetryMiddleware(
            max_retries=3,
            backoff_factor=2,
            retryable_errors=["RateLimit", "InternalError"],
        ),
    ],
)
```

配置选项
  "max_retries" type="number" default="3
    失败模型调用的最大重试次数。
  

  "backoff_factor" type="float" default="2
    重试之间的指数退避因子。
  

  "retryable_errors" type="list[string]
    应重试的错误类型列表。如果未指定，重试所有错误。
  


### LLM工具模拟器

使用LLM模拟工具执行以进行测试，无需实际调用外部API。LLM工具模拟器在以下场景中非常有用：

* 开发和测试期间的工具模拟
* 成本敏感的开发环境
* 需要隔离测试的CI/CD流水线

> **API参考：** [`LLMToolEmulatorMiddleware`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.LLMToolEmulatorMiddleware)

```python
from langchain.agents import create_agent
from langchain.agents.middleware import LLMToolEmulatorMiddleware

agent = create_agent(
    model="gpt-4o",
    tools=[search_tool, calculator_tool],
    middleware=[
        LLMToolEmulatorMiddleware(
            emulator_model="gpt-4o-mini",
            simulated_tools=["search"],
            realism_level="high",
        ),
    ],
)
```

配置选项
  "emulator_model" type="string | BaseChatModel" required
    用于工具模拟的模型。可以是模型标识符字符串（例如`'openai:gpt-4o-mini'`）或`BaseChatModel`实例。
  

  "simulated_tools" type="list[string]" required
    要模拟的工具名称列表。
  

  "realism_level" type="string" default="medium
    模拟的真实性级别：`'low'`、`'medium'`或`'high'`。
  

  "emulator_prompt" type="string
    工具模拟的自定义提示模板。
  


### 上下文编辑

通过修剪或清除工具使用来管理对话上下文，优化token使用。上下文编辑在以下场景中非常有用：

* 需要优化token使用的长时间运行对话
* 需要清理工具调用历史的应用
* 管理上下文窗口有限的模型

> **API参考：** [`ContextEditingMiddleware`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.ContextEditingMiddleware)

```python
from langchain.agents import create_agent
from langchain.agents.middleware import ContextEditingMiddleware

agent = create_agent(
    model="gpt-4o",
    tools=[search_tool, calculator_tool],
    middleware=[
        ContextEditingMiddleware(
            strategy="trim",
            max_tool_uses=5,
            preserve_recent=True,
        ),
    ],
)
```

配置选项
  "strategy" type="string" default="trim
    上下文编辑策略：`'trim'`（修剪旧工具使用）或`'clear'`（清除所有工具使用）。
  

  "max_tool_uses" type="number" default="10
    保留的最大工具使用数（仅适用于`trim`策略）。
  

  "preserve_recent" type="boolean" default="true
    是否保留最近的工具使用（仅适用于`trim`策略）。
  

  "trigger" type="ContextSize | list[ContextSize] | None
    触发上下文编辑的条件。类似于摘要生成的触发条件。
  


### Shell工具

为Agent暴露持久shell会话以执行命令，允许Agent与本地系统交互。Shell工具在以下场景中非常有用：

* 需要系统级操作的自动化脚本
* 文件系统管理和操作
* 命令行工具集成

> **API参考：** [`ShellToolMiddleware`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.ShellToolMiddleware)

```python
from langchain.agents import create_agent
from langchain.agents.middleware import ShellToolMiddleware

agent = create_agent(
    model="gpt-4o",
    tools=[],
    middleware=[
        ShellToolMiddleware(
            working_dir="/tmp",
            allowed_commands=["ls", "cat", "grep"],
            timeout=30,
        ),
    ],
)
```

配置选项
  "working_dir" type="string" default="/tmp
    shell会话的工作目录。
  

  "allowed_commands" type="list[string]
    允许执行的命令列表。如果未指定，允许所有命令（谨慎使用）。
  

  "timeout" type="number" default="30
    命令执行的超时时间（秒）。
  

  "persistent" type="boolean" default="true
    是否维护持久shell会话。
  


### 文件搜索

提供文件系统文件的Glob和Grep搜索工具，允许Agent搜索和读取本地文件。文件搜索在以下场景中非常有用：

* 文档检索和分析
* 代码库搜索
* 日志文件分析

> **API参考：** [`FileSearchMiddleware`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.FileSearchMiddleware)

```python
from langchain.agents import create_agent
from langchain.agents.middleware import FileSearchMiddleware

agent = create_agent(
    model="gpt-4o",
    tools=[],
    middleware=[
        FileSearchMiddleware(
            search_dirs=["/home/user/documents"],
            max_file_size=1024 * 1024,  # 1MB
            allowed_extensions=[".txt", ".md", ".py"],
        ),
    ],
)
```

配置选项
  "search_dirs" type="list[string]" required
    要搜索的目录列表。
  

  "max_file_size" type="number" default="1048576
    要搜索的最大文件大小（字节）。
  

  "allowed_extensions" type="list[string]
    允许搜索的文件扩展名列表。如果未指定，允许所有扩展名。
  

  "recursive" type="boolean" default="true
    是否递归搜索子目录。
  


## 提供商特定中间件

某些中间件专为特定LLM提供商设计：

* [Anthropic特定中间件](/oss/python/langchain/anthropic#middleware)
* [OpenAI特定中间件](/oss/python/langchain/openai#middleware)

有关提供商特定中间件的完整列表，请参阅相应的文档。
```